# Section 2.2 Data Structure Activities

In [1]:
import os
import pymc3 as pm
import arviz as az
import pandas as pd
import numpy as np

if os.path.split(os.getcwd())[-1] != "notebooks":
    os.chdir(os.path.join(".."))
    
np.random.seed(0)

In [2]:
az.style.use('arviz-white')

## Reproducing the Planet Experiment
More good news! Your astronomical discovery, from Section 1.2, has been published, but now people want to you to share your data and results. They also are asking for help getting seeing portions of your analysis runs to inspect in greater detail.

### Exercise 1 
Your favorite PPL is PyMC3, but it turns out your peer reviewer likes Stan. In an alternate universe your favorite PPL is Stan, but now your peer reviewer is a PyMC3 gal. Here we introduce the *Law of researcher PPL choice*  

$$P(\text{Your friends uses another PPL} | \text{You choice of PPL}) = 1$$


**How can we use ArviZ, Xarray, and NetCDF to share results in a common way?**  

Note: We encourage you to use whatever PPL you prefer. These docs may be helpful:

https://arviz-devs.github.io/arviz/api.html#data

#### Step 1: Define your model and generate results
The first step you'll need to take is to define your model and perform an inference run in this notebook. Recall in mathematical notation our model was as follows

$$\theta = \text{Proportion of water on the planet}$$

$$ 
\theta \sim \operatorname{Uniform}(0,1) \\
p_{\text{water}} \sim \operatorname{Binom}(\theta, N)
$$

In [3]:
observations = [0, 0, 1, 0, 1]
water_observations = sum(observations)
total_observations = len(observations)

In [4]:
# Add your model here, and perform an inference run. 
# You can either copy the model from Section 1.2 or for a challenge define your own
with pm.Model() as planet_model:
    p_water = pm.Uniform("p", 0 ,1)
    w = pm.Binomial("w", p=p_water, n=total_observations, observed=water_observations)
    trace = pm.sample(5000, chains=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [p]
100%|██████████| 5500/5500 [00:03<00:00, 1451.05it/s]


#### Step 2: Convert model results from PPL to az.InferenceData

In [5]:
# Hint: Use the az.from_ family of methods
water_data = az.from_pymc3(trace=trace)

#### Step 3: Inspect InferenceData to see what groups exist

In [6]:
# Inspect groups here
water_data

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

#### Step 4: Inspect Posterior group to verify variables count, chain count, and draw count

In [7]:
# Inspect Posterior here
water_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 2, draw: 5000)
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 4993 4994 4995 4996 4997 4998 4999
Data variables:
    p        (chain, draw) float64 0.2813 0.3035 0.1876 ... 0.5252 0.4642 0.4642
Attributes:
    created_at:                 2019-10-12T04:55:07.503254
    inference_library:          pymc3
    inference_library_version:  3.6

In [8]:
# Inspect sample_stats here
water_data.sample_stats

<xarray.Dataset>
Dimensions:               (chain: 2, draw: 5000, log_likelihood_dim_0: 1)
Coordinates:
  * chain                 (chain) int64 0 1
  * draw                  (draw) int64 0 1 2 3 4 5 ... 4995 4996 4997 4998 4999
  * log_likelihood_dim_0  (log_likelihood_dim_0) int64 0
Data variables:
    lp                    (chain, draw) float64 -2.824 -2.721 ... -2.496 -2.496
    tree_size             (chain, draw) float64 1.0 1.0 3.0 3.0 ... 1.0 3.0 1.0
    max_energy_error      (chain, draw) float64 0.0315 -0.03656 ... -0.068 1.902
    energy                (chain, draw) float64 2.911 2.868 ... 2.559 4.171
    energy_error          (chain, draw) float64 0.0315 -0.03656 ... -0.068 0.0
    step_size_bar         (chain, draw) float64 1.171 1.171 ... 1.284 1.284
    tune                  (chain, draw) bool False False False ... False False
    diverging             (chain, draw) bool False False False ... False False
    mean_tree_accept      (chain, draw) float64 0.969 1.0 ... 0.9941 

In [9]:
# Inspect observed_data here
water_data.observed_data

<xarray.Dataset>
Dimensions:  (w_dim_0: 1)
Coordinates:
  * w_dim_0  (w_dim_0) int64 0
Data variables:
    w        (w_dim_0) float64 2.0
Attributes:
    created_at:                 2019-10-12T04:55:08.355507
    inference_library:          pymc3
    inference_library_version:  3.6

#### Step 3: Save your model to disk

In [10]:
# Save your results here
water_data.to_netcdf("WaterResults.nc")

'WaterResults.nc'

### Exercise 2
You've been asked to peer review a study on radon levels in Minnesota basements. The dataset is available as part of ArviZ's remote datasets. You've been asked to do a couple things.

#### Step 1: Load the NetCDF file into python memory
*Note*: In ArviZ there are preloaded datasets. Radon is one of those. If you do not supply a dataset name to the correct method, an error message lists all the available ones.

In [11]:
# Load Radon dataset. Hint look for the load_arviz_data
radon_data = az.load_arviz_data(dataset="radon")

#### Step 2: List all the groups
See what analysis your colleague has already run by checking the groups present in the InferenceData object

In [12]:
# Inspect groups here
radon_data

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data

#### Step 3: Count the number of counties included in radon study
How many counties were included in the observed_data?
Hint: xarray has a `.to_dataframe()` method

In [13]:
type(radon_data.observed_data.to_dataframe())

pandas.core.frame.DataFrame

In [14]:
# Use xarray and pandas index to determine number of counties
radon_data.observed_data.to_dataframe().head(10)

,y_like
observed_county,
AITKIN,0.832909
AITKIN,0.832909
AITKIN,1.098612
AITKIN,0.095310
ANOKA,1.163151
ANOKA,0.955511
ANOKA,0.470004
ANOKA,0.095310
ANOKA,-0.223144


In [15]:
radon_data.observed_data.to_dataframe().index.unique()[:10]

Index(['AITKIN', 'ANOKA', 'BECKER', 'BELTRAMI', 'BENTON', 'BIG STONE',
       'BLUE EARTH', 'BROWN', 'CARLTON', 'CARVER'],
      dtype='object', name='observed_county')

In [16]:
radon_data.observed_data.to_dataframe().index.unique().shape

(85,)

#### How many variables are in Bayesian model?
Inspect the posterior xarray dataset and get a list of data variables in xarray.

In [17]:
# Use xarray to see how many data variables exist
radon_data.posterior

<xarray.Dataset>
Dimensions:          (chain: 4, county: 85, draw: 500, gamma_dim_0: 3, observed_county: 919)
Coordinates:
  * chain            (chain) int64 0 1 2 3
  * draw             (draw) int64 0 1 2 3 4 5 6 ... 493 494 495 496 497 498 499
  * gamma_dim_0      (gamma_dim_0) int64 0 1 2
  * county           (county) object 'AITKIN' 'ANOKA' ... 'YELLOW MEDICINE'
  * observed_county  (observed_county) object 'AITKIN' ... 'YELLOW MEDICINE'
Data variables:
    gamma            (chain, draw, gamma_dim_0) float64 ...
    eps_a            (chain, draw, county) float64 ...
    b                (chain, draw) float64 ...
    sigma_a          (chain, draw) float64 ...
    mu_a             (chain, draw, observed_county) float64 ...
    a                (chain, draw, observed_county) float64 ...
    sigma_y          (chain, draw) float64 ...
Attributes:
    created_at:                 2018-10-05T15:29:14.514378
    inference_library:          pymc3
    inference_library_version:  3.5

#### Step 4: Select the first 10 values of chain 2 for sigma_y in the posterior
Using the `.sel` method get the first ten values 

In [18]:
# Get First ten values
radon_data.posterior.sel(chain=[2], draw=slice(0,10))["sigma_y"]

<xarray.DataArray 'sigma_y' (chain: 1, draw: 11)>
array([[0.748223, 0.748223, 0.738897, 0.742252, 0.749223, 0.743694, 0.760952,
        0.760952, 0.760952, 0.760952, 0.711894]])
Coordinates:
  * chain    (chain) int64 2
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 9 10